In [2]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb rank_bm25 faiss-cpu sentence-transformers
!pip install -U langchain-community
!pip install langchain-huggingface
!pip install pythainlp
!pip install python-crfsuite
!pip install stop_words
!pip install FlagEmbedding
!pip install datasets
!pip install rank-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.0 M

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from FlagEmbedding import BGEM3FlagModel

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
dataset = load_dataset("chompk/tydiqa-goldp-th")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for chompk/tydiqa-goldp-th contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/chompk/tydiqa-goldp-th.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [5]:
all_doc = set(dataset['validation']['context'])
all_doc = {c:i for i,c in enumerate(all_doc)}

question_contextid_context = []
for item in dataset['validation']:
    question = item['question']
    doc = item['context']
    question_contextid_context.append([all_doc[doc],question])

df_question = pd.DataFrame(question_contextid_context, columns =['doc_id','question'])
df_document = pd.DataFrame(zip(list(all_doc.values()),list(all_doc.keys())), columns =['doc_id','document'])

# Vector search

In [6]:
#@title Choose Pretrained Model
model_name = "bge-m3" #@param ["simcse-model-wangchanberta", "bge-m3", "paraphrase-multilingual-mpnet-base-v2", "ConGen-BGE_M3-model-phayathaibert"]

if model_name == "bge-m3":
  embed = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
  def embed_encode(docs):
    return embed.encode(docs)['dense_vecs']

elif model_name == "simcse-model-wangchanberta":
  embed = SentenceTransformer("kornwtp/simcse-model-wangchanberta")
  def embed_encode(docs):
    return embed.encode(docs,convert_to_numpy=True,normalize_embeddings=True)

elif model_name == "paraphrase-multilingual-mpnet-base-v2":
  embed = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
  def embed_encode(docs):
    return embed.encode(docs,convert_to_numpy=True,normalize_embeddings=True)

elif model_name ==  "ConGen-BGE_M3-model-phayathaibert":
  embed = SentenceTransformer("kornwtp/ConGen-BGE_M3-model-phayathaibert")
  def embed_encode(docs):
    return embed.encode(docs,convert_to_numpy=True,normalize_embeddings=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

imgs/bm25.jpg:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

...

imgs/long.jpg:   0%|          | 0.00/485k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [7]:
import numpy as np
def evaluate(question_id,question_all,context_id,context_all,mrr_rank=10,status=True):
    top_1 = 0; top_5 = 0; top_10 = 0;
    mrr_score = 0
    context_id = np.array(context_id)
    sim_score = np.inner(question_all,context_all)
    print(sim_score.shape)
    status_bar = enumerate(sim_score)
    for idx,sim in status_bar:
        index = np.argsort(sim)[::-1]
        index_edit = [context_id[x] for x in index]
        idx_search = list(index_edit).index(question_id[idx])
        if idx_search == 0:
            top_1+=1
            top_5+=1
            top_10+=1
        elif idx_search < 5:
            top_5+=1
            top_10+=1
        elif idx_search < 10:
            top_10+=1
        if idx_search < mrr_rank:
            mrr_score += (1/(idx_search+1))
    mrr_score/=len(question_all)

    precision = top_1 / len(question_all)
    print(f"Traninng Score P@1: {precision:.4f}")
    precision = top_5 / len(question_all)
    print(f"Traninng Score P@5: {precision:.4f}")
    precision = top_10 / len(question_all)
    print(f"Traninng Score P@10: {precision:.4f}")
    print(f"Mrr score:{mrr_score:.4f}")

    return top_1,top_5,top_10,mrr_score,sim_score

In [8]:
doc_context_id = df_document['doc_id'].to_list()
doc_context_encoded = embed_encode(df_document['document'].tolist())

question_id = df_question['doc_id'].to_list()
questions = embed_encode(df_question['question'].tolist())

Inference Embeddings: 100%|██████████| 64/64 [00:01<00:00, 44.57it/s]


In [9]:
top_1,top_5,top_10,mrr,sim_score = evaluate(question_id,questions,doc_context_id,doc_context_encoded)

(763, 668)
Traninng Score P@1: 0.8912
Traninng Score P@5: 0.9869
Traninng Score P@10: 0.9934
Mrr score:0.9343


# RAG LLM

In [74]:
df_question['question'][8]

'ธรณีวิทยา ศึกษาเกี่ยวกับอะไร ?'

In [10]:
def rank_vector(query,doc_encode,docs):
  sim_score = np.inner(embed_encode(query) , doc_encode)
  index = np.argsort(sim_score)[::-1]
  return zip(index[:5],[re.sub('\r?\n', ' ', i) for i in list(docs[list(index[:5])])])

In [75]:
relevent = rank_vector(df_question['question'][8], doc_context_encoded, df_document['document'])
relevent_text = '{ '
for idx,i in relevent:
  relevent_text += f'{idx}: "{i}", '
  print(i)
relevent_text = relevent_text[:-2]
relevent_text += ' }'
# relevent_text

ธรณีวิทยา, ธรณีศาสตร์ (English: Geology จาก[ γη- (เก-, โลก) และ λογος (ลอกอส, ถ้อยคำ หรือ เหตุผล)]error: {{lang-xx}}: text has italic markup (help)) เป็นวิทยาศาสตร์ที่ศึกษาเกี่ยวกับโลก สสารต่าง ๆ ที่เป็นส่วนประกอบของโลก เช่น แร่ หิน ดินและน้ำ รวมทั้งกระบวนการเปลี่ยนแปลงภายในโลก ที่เกิดขึ้นในธรรมชาติ ตั้งแต่กำเนิดโลกจนถึงปัจจุบัน เป็นการศึกษาทั้งในระดับโครงสร้าง ส่วนประกอบทางกายภาพ เคมี และชีววิทยา ทำให้รู้ถึงประวัติความเป็นมา และสภาวะแวดล้อมในอดีตจนถึงปัจจุบัน ศึกษาปัจจัยต่าง ๆ ทั้งภายใน และภายนอกที่มีอิทธิพลต่อการเปลี่ยนแปลงสภาพพื้นผิว วิวัฒนาการของสิ่งมีชีวิต ตลอดจนรูปแบบ และวิธีการนำเอาทรัพยากรธรรมชาติ มาใช้ประโยชน์อย่างยั่งยืนอีกด้วย
ดาราศาสตร์ คือวิชาวิทยาศาสตร์ที่ศึกษาวัตถุท้องฟ้า (อาทิ ดาวฤกษ์ ดาวเคราะห์ ดาวหาง และดาราจักร) รวมทั้งปรากฏการณ์ทางธรรมชาติต่าง ๆ ที่เกิดขึ้นจากนอกชั้นบรรยากาศของโลก โดยศึกษาเกี่ยวกับวิวัฒนาการ ลักษณะทางกายภาพ ทางเคมี ทางอุตุนิยมวิทยา และการเคลื่อนที่ของวัตถุท้องฟ้า ตลอดจนถึงการกำเนิดและวิวัฒนาการของเอกภพ
นรีเวชศาสตร์ หรือ นรีเวชวิทยา หรือ วิทยาเพศหญิง

In [15]:
import os
from openai import OpenAI

os.environ["OPENTYPHOON_API_KEY"] = "OPENTYPHOON_API_KEY"
client = OpenAI(api_key=os.environ["OPENTYPHOON_API_KEY"], base_url="https://api.opentyphoon.ai/v1")

In [76]:
ex_doc = '{1: "ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่นทาง โดยสร้างโดย TOHO studio", 2:"ทาง Muse Thailand นำเข้าอนิเมะซีซันที่ 1 ออกอากาศหลังทางญี่ปุ่น 7 วัน ที่ไทย", 3:"หมาบินไม่ได้นะ เด็กๆ", 4: "ส่วนทางซีซันที่ 2 ของไคจูหมายเลข8 มีแผนจะฉายในปีหน้า ที่ญี่ปุ่นที่แรก", 5:"the ghost radio เป็นช่องที่เล่าเรื่องผีที่ยอดนิยม"}'
ex_q = 'ไคจูหมายเลข8ซีซั่นแรกออกอากาศเมื่อวันที่เท่าไหร่?'
ex_aws = 'ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่น และออกอากาศในวันที่ 14 เมษายน 2567 ทางประเทศไทย [1],[2]'
sys_promt = f"You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:\nKeep your answer ground in the facts provided in REAL DOCUMENT section and insert REAL DOCUMENTS ID that support your REAL ANSWER.\nIf the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].\nFor example:\nEXAMPLE DOCUMENT: {ex_doc}\nEXAMPLE QUESTION: {ex_q}\nEXAMPLE ANSWER: {ex_aws}"
query_promt = 'REAL DOCUMENT: {}\nREAL QUESTION: {}\nREAL ANSWER:'.format(relevent_text,df_question['question'][8])
print(sys_promt)
print(query_promt)

You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:
Keep your answer ground in the facts provided in REAL DOCUMENT section and insert REAL DOCUMENTS ID that support your REAL ANSWER.
If the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].
For example:
EXAMPLE DOCUMENT: {1: "ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่นทาง โดยสร้างโดย TOHO studio", 2:"ทาง Muse Thailand นำเข้าอนิเมะซีซันที่ 1 ออกอากาศหลังทางญี่ปุ่น 7 วัน ที่ไทย", 3:"หมาบินไม่ได้นะ เด็กๆ", 4: "ส่วนทางซีซันที่ 2 ของไคจูหมายเลข8 มีแผนจะฉายในปีหน้า ที่ญี่ปุ่นที่แรก", 5:"the ghost radio เป็นช่องที่เล่าเรื่องผีที่ยอดนิยม"}
EXAMPLE QUESTION: ไคจูหมายเลข8ซีซั่นแรกออกอากาศเมื่อวันที่เท่าไหร่?
EXAMPLE ANSWER: ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่น และออกอากาศในวันที่ 14 เมษายน 2567 ทางประเทศไทย [1],[2]
REAL DOCUMENT: { 492: "ธรณีวิทยา, ธรณีศาสตร์ (English: Geology จาก[ γη- (เก-, โลก) และ λογος (ลอกอส, ถ้อยคำ ห

In [77]:
response = client.chat.completions.create(
    model="typhoon-instruct",
    messages=[{"role": "system", "content": sys_promt},{"role": "user", "content": query_promt}],
    max_tokens=500,
    temperature=0.7,
    top_p=1,
)

response.choices[0].message.content

'ธรณีวิทยา, ธรณีศาสตร์ ศึกษาเกี่ยวกับโลก สสารต่าง ๆ ที่เป็นส่วนประกอบของโลก เช่น แร่ หิน ดินและน้ำ รวมทั้งกระบวนการเปลี่ยนแปลงภายในโลก ที่เกิดขึ้นในธรรมชาติ ตั้งแต่กำเนิดโลกจนถึงปัจจุบัน เป็นการศึกษาทั้งในระดับโครงสร้าง ส่วนประกอบทางกายภาพ เคมี และชีววิทยา ทำให้รู้ถึงประวัติความเป็นมา และสภาวะแวดล้อมในอดีตจนถึงปัจจุบัน ศึกษาปัจจัยต่าง ๆ ทั้งภายใน และภายนอกที่มีอิทธิพลต่อการเปลี่ยนแปลงสภาพพื้นผิว วิวัฒนาการของสิ่งมีชีวิต ตลอดจนรูปแบบ และวิธีการนำเอาทรัพยากรธรรมชาติ มาใช้ประโยชน์อย่างยั่งยืน [492]'

#  lexical search

In [78]:
import nltk

# Download the necessary resources
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize as thai_word_tokenize
from collections import Counter

en_stop = set(stopwords.words('english'))
th_stop = set(thai_stopwords())

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [79]:
# Initialize stemmer and lemmatizer

p_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def clean(text):
  text = text.translate(str.maketrans('', '', '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~'''))
  text =text.lower()
  text = text.replace('\n', '')
  text = ' '.join(text.split())
  return text

def split_word_iterative(tokens):
    result = []
    for token in tokens:
        # Clean the token
        token = clean(token)

        # Remove stop words in Thai and English
        if token not in th_stop and token not in en_stop:
            # Stem English words
            token = p_stemmer.stem(token)

            # Find the Thai word roots using WordNet
            w_syn = wordnet.synsets(token)
            if len(w_syn) > 1 and len(w_syn[0].lemma_names('tha')) > 1:
                token = w_syn[0].lemma_names('tha')[0]

            # Remove numbers
            if not token.isnumeric():
                # Remove spaces
                if ' ' not in token and any(token) and '"' not in token:
                    result.append(token)
    return result

def split_word(text):
    # Tokenize the text
    tokens = thai_word_tokenize(text, engine='newmm')

    # Iteratively process tokens
    return split_word_iterative(tokens)

In [80]:
tokenized_context = [split_word(doc) for doc in df_document['document'].tolist()]
doc_context_id = df_document['doc_id'].to_list()

question_id = df_question['doc_id'].to_list()
tokenized_question = [split_word(q) for q in df_question['question'].tolist()]

In [81]:
from rank_bm25 import BM25Okapi

def evaluate_b25(question_id,tokenized_question,context_id,tokenized_context,mrr_rank=10,status=True):
    top_1 = 0; top_5 = 0; top_10 = 0;
    mrr_score = 0
    context_id = np.array(context_id)
    score =[]
    bm25 = BM25Okapi(tokenized_context)
    for q in tokenized_question:
      doc_scores = bm25.get_scores(q)
      score.append(doc_scores)
    sim_score = np.array(score)
    print(sim_score.shape)
    status_bar = enumerate(sim_score)
    for idx,sim in status_bar:
        index = np.argsort(sim)[::-1]
        index_edit = [context_id[x] for x in index]
        idx_search = list(index_edit).index(question_id[idx])
        if idx_search == 0:
            top_1+=1
            top_5+=1
            top_10+=1
        elif idx_search < 5:
            top_5+=1
            top_10+=1
        elif idx_search < 10:
            top_10+=1
        if idx_search < mrr_rank:
            mrr_score += (1/(idx_search+1))
    mrr_score/=len(question_id)

    precision = top_1 / len(question_id)
    print(f"Traninng Score P@1: {precision:.4f}")
    precision = top_5 / len(question_id)
    print(f"Traninng Score P@5: {precision:.4f}")
    precision = top_10 / len(question_id)
    print(f"Traninng Score P@10: {precision:.4f}")
    print(f"Mrr score:{mrr_score:.4f}")

    return top_1,top_5,top_10,mrr_score,sim_score

In [82]:
top_1,top_5,top_10,mrr,sim_score = evaluate_b25(question_id,tokenized_question,doc_context_id,tokenized_context)

(763, 668)
Traninng Score P@1: 0.7641
Traninng Score P@5: 0.9135
Traninng Score P@10: 0.9358
Mrr score:0.8287


#  lexical RAG LLM

In [83]:
def rank_b25(token_query,token_doc,docs):
  bm25 = BM25Okapi(token_doc)
  sim_score = bm25.get_scores(token_query)
  index = np.argsort(sim_score)[::-1]
  return zip(index[:5],[re.sub('\r?\n', ' ', i) for i in list(docs[list(index[:5])])])

In [84]:
relevent = rank_b25(tokenized_question[8], tokenized_context, df_document['document'])
relevent_text = '{ '
for idx,i in relevent:
  relevent_text += f'{idx}: "{i}", '
  print(i)
relevent_text = relevent_text[:-2]
relevent_text += ' }'
# relevent_text

ธรณีวิทยา, ธรณีศาสตร์ (English: Geology จาก[ γη- (เก-, โลก) และ λογος (ลอกอส, ถ้อยคำ หรือ เหตุผล)]error: {{lang-xx}}: text has italic markup (help)) เป็นวิทยาศาสตร์ที่ศึกษาเกี่ยวกับโลก สสารต่าง ๆ ที่เป็นส่วนประกอบของโลก เช่น แร่ หิน ดินและน้ำ รวมทั้งกระบวนการเปลี่ยนแปลงภายในโลก ที่เกิดขึ้นในธรรมชาติ ตั้งแต่กำเนิดโลกจนถึงปัจจุบัน เป็นการศึกษาทั้งในระดับโครงสร้าง ส่วนประกอบทางกายภาพ เคมี และชีววิทยา ทำให้รู้ถึงประวัติความเป็นมา และสภาวะแวดล้อมในอดีตจนถึงปัจจุบัน ศึกษาปัจจัยต่าง ๆ ทั้งภายใน และภายนอกที่มีอิทธิพลต่อการเปลี่ยนแปลงสภาพพื้นผิว วิวัฒนาการของสิ่งมีชีวิต ตลอดจนรูปแบบ และวิธีการนำเอาทรัพยากรธรรมชาติ มาใช้ประโยชน์อย่างยั่งยืนอีกด้วย
ดร.จักรภพ เพ็ญแข เกิดเมื่อวันที่ 21 ตุลาคม พ.ศ. 2510 ศึกษาชั้นประถมศึกษา และมัธยมศึกษา ที่โรงเรียนสาธิตแห่งมหาวิทยาลัยเกษตรศาสตร์ จากนั้น เข้าศึกษาต่อที่คณะรัฐศาสตร์ สาขาความสัมพันธ์ระหว่างประเทศ จุฬาลงกรณ์มหาวิทยาลัย เมื่อจบปริญญาตรีแล้ว จึงเข้าศึกษาต่อระดับปริญญาโทและปริญญาเอก ที่ The Paul H. Nitze School of Advanced International Studies (SAIS) มหาว

In [85]:
import os
from openai import OpenAI

os.environ["OPENTYPHOON_API_KEY"] = "OPENTYPHOON_API_KEY"
client = OpenAI(api_key=os.environ["OPENTYPHOON_API_KEY"], base_url="https://api.opentyphoon.ai/v1")

In [86]:
ex_doc = '{1: "ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่นทาง โดยสร้างโดย TOHO studio", 2:"ทาง Muse Thailand นำเข้าอนิเมะซีซันที่ 1 ออกอากาศหลังทางญี่ปุ่น 7 วัน ที่ไทย", 3:"หมาบินไม่ได้นะ เด็กๆ", 4: "ส่วนทางซีซันที่ 2 ของไคจูหมายเลข8 มีแผนจะฉายในปีหน้า ที่ญี่ปุ่นที่แรก", 5:"the ghost radio เป็นช่องที่เล่าเรื่องผีที่ยอดนิยม"}'
ex_q = 'ไคจูหมายเลข8ซีซั่นแรกออกอากาศเมื่อวันที่เท่าไหร่?'
ex_aws = 'ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่น และออกอากาศในวันที่ 14 เมษายน 2567 ทางประเทศไทย [1],[2]'
sys_promt = f"You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:\nKeep your answer ground in the facts provided in REAL DOCUMENT section and insert REAL DOCUMENTS ID that support your REAL ANSWER.\nIf the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].\nFor example:\nEXAMPLE DOCUMENT: {ex_doc}\nEXAMPLE QUESTION: {ex_q}\nEXAMPLE ANSWER: {ex_aws}"
query_promt = 'REAL DOCUMENT: {}\nREAL QUESTION: {}\nREAL ANSWER:'.format(relevent_text,df_question['question'][8])
print(sys_promt)
print(query_promt)

You are a helpful assistant that must try your best effort to answer the user question ALWAYS following this guidelines:
Keep your answer ground in the facts provided in REAL DOCUMENT section and insert REAL DOCUMENTS ID that support your REAL ANSWER.
If the DOCUMENT section doesn’t contain the facts to answer the QUESTION ALWAYS return [NONE].
For example:
EXAMPLE DOCUMENT: {1: "ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่นทาง โดยสร้างโดย TOHO studio", 2:"ทาง Muse Thailand นำเข้าอนิเมะซีซันที่ 1 ออกอากาศหลังทางญี่ปุ่น 7 วัน ที่ไทย", 3:"หมาบินไม่ได้นะ เด็กๆ", 4: "ส่วนทางซีซันที่ 2 ของไคจูหมายเลข8 มีแผนจะฉายในปีหน้า ที่ญี่ปุ่นที่แรก", 5:"the ghost radio เป็นช่องที่เล่าเรื่องผีที่ยอดนิยม"}
EXAMPLE QUESTION: ไคจูหมายเลข8ซีซั่นแรกออกอากาศเมื่อวันที่เท่าไหร่?
EXAMPLE ANSWER: ซีซันที่ 1 ออกอากาศเมื่อ 9 เมษายน 2567 ที่ญี่ปุ่น และออกอากาศในวันที่ 14 เมษายน 2567 ทางประเทศไทย [1],[2]
REAL DOCUMENT: { 492: "ธรณีวิทยา, ธรณีศาสตร์ (English: Geology จาก[ γη- (เก-, โลก) และ λογος (ลอกอส, ถ้อยคำ ห

In [87]:
response = client.chat.completions.create(
    model="typhoon-instruct",
    messages=[{"role": "system", "content": sys_promt},{"role": "user", "content": query_promt}],
    max_tokens=500,
    temperature=0.7,
    top_p=1,
)

response.choices[0].message.content

'ธรณีวิทยา ศึกษาเกี่ยวกับโลก, สสารต่าง ๆ ที่เป็นส่วนประกอบของโลก เช่น แร่, หิน, ดินและน้ำ รวมทั้งกระบวนการเปลี่ยนแปลงภายในโลก ที่เกิดขึ้นในธรรมชาติ, ตั้งแต่กำเนิดโลกจนถึงปัจจุบัน โดยเป็นการศึกษาทั้งในระดับโครงสร้าง, ส่วนประกอบทางกายภาพ, เคมี และชีววิทยา [492]'